In [31]:

# Questa funzione legge i file della oc_chamber
def reader(percorso_al_file):
    
    f = open(percorso_al_file)

    dati = dict() # struttura dati a dizionario in cui le chiavi sono le colonne e i valori sono stored in una lista
    dati["Event"] = list()
    dati["Answer"] = list()
    dati["RT"] = list()
    dati["Time"] = list()

    for line in f:
        to_list = line.split("\t")
        #row = list()
        for col in to_list:
            value = col.split(": ")
            if value[0] == "RT":
                value[1] = int(value[1])
            if value[0] == "Time":
                value[1] = value[1][:-1]

            dati[value[0]].append(value[1])

    f.close()
    
    return dati

# calcola il numero totale e parziale di trial
def ntrial(dati):
    total = dict()
    total["total"] = 0
    total["right"] = 0
    total["left"] = 0

    for r in dati["Event"]:
        if r=="left":
            total["left"] = total["left"]+1
        elif r=="right":
            total["right"] = total["right"]+1
        total["total"] = total["total"] +1
            
    return total


In [13]:

# contare numero di trial
# contare il tipo di trial
# tempo di reazione medio
# correttezza percentuale totale
# tempo di reazione medio per tipo di trial
# correttezza percentuale per tipo di trial 
# calcolare intertrial
# creare libreria di funzioni che calcola le robe fatte su



In [14]:

percorso = "C:\\Users\\oldboy\\Documents\\oc_chamber\\dataset\\data\\F-B00-WT\\"

percorsofile = percorso + "20180116_0948-FB00.txt"
dati = reader(percorsofile)



In [19]:

print("Numero di trial: " , ntrial(dati))

Numero di trial:  30


In [25]:

n_left = 0
n_right = 0
for r in dati["Event"]:
    if r=="left":
        n_left = n_left+1
    elif r=="right":
        n_right = n_right+1

print("Left:", n_left,"\tRight: ",n_right)

Left: 28 	Right:  2


In [32]:



total = ntrial(dati)
print("\tTotal:", total["total"],"\tLeft:", total["left"],"\tRight: ",total["right"])


	Total: 30 	Left: 28 	Right:  2
